## This notebook works
## I was able to train to 0.084 loss
## Train it again and try to get lowest loss possible

Here is the dataset as a pickle file

In [13]:
# subsetted dataset
# this is in the shared drive in data folder
filtered_dataset_path = '/content/drive/My Drive/Colab Notebooks/NLP/Machine Translation/FT_Files/pairs.pkl'

Paths to WEs

In [14]:
# ft vectors
# this is in shared drive in ft folder
ft_path = '/content/drive/My Drive/Colab Notebooks/NLP/Machine Translation/FT_Files/wiki-news-300d-1M.vec'

#frWac vectors
# this is in shared drive in glove_frwac folder
frWac200d_path = '/content/drive/My Drive/Colab Notebooks/NLP/Machine Translation/FT_Files/cc.fr.300_fasttext_french.vec'

# model weights
# you won't have these till you train the model
encoder_saved_model_weights = '/content/drive/My Drive/Colab Notebooks/NLP/Machine Translation/FT_Files/FTsimple_encoder_200E_0.084Loss.pth'
decoder_saved_model_weights ='/content/drive/My Drive/Colab Notebooks/NLP/Machine Translation/FT_Files/FTsimple_decoder_200E_0.084Loss.pth'

In [15]:
!pip install bcolz

In [16]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


import itertools 
import os, re, pickle, collections, bcolz, string
import numpy as np
import math
import gensim
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook
from gensim.models import KeyedVectors
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load and separate sentences

In [18]:
sentences = pickle.load(open(filtered_dataset_path, 'rb'))

In [19]:
en_questions, fr_questions = zip(*sentences)

Define tokenizer.

In [20]:
re_apos = re.compile(r"(\w)'s\b")         # make 's a separate word
re_mw_punc = re.compile(r"(\w[’'])(\w)")  # other ' in a word creates 2 words
re_punc = re.compile("([\"().,;:/_?!—])") # add spaces around punctuation
re_mult_space = re.compile(r"  *")        # replace multiple spaces with just one

def tokenize(sent):
    sent = re_apos.sub(r"\1 's", sent)
    sent = re_mw_punc.sub(r"\1 \2", sent)
    sent = re_punc.sub(r" \1 ", sent).replace('-', ' ')
    sent = re_mult_space.sub(' ', sent)
    return sent.lower().split()

Tokenize english questions.

In [21]:
en_tokens = list(map(tokenize, en_questions))

In [22]:
en_tokens[:4]

[['i', 'm', '.'],
 ['i', 'm', 'ok', '.'],
 ['i', 'm', 'ok', '.'],
 ['i', 'm', 'fat', '.']]

Tokenize french questions.

In [23]:
fr_tokens = list(map(tokenize, fr_questions))

In [24]:
fr_tokens[:4]

[['j', 'ai', 'ans', '.'],
 ['je', 'vais', 'bien', '.'],
 ['ca', 'va', '.'],
 ['je', 'suis', 'gras', '.']]

For each language: 
<br/>- Get vocabulary counter.
<br/>- Get vocabulary.
<br/>- Get dictionary that maps each word to an index.
<br/>- Transform tokens to their corresponding ids.

In [25]:
PAD = 0; SOS = 1

def tokens2ids(sentences):
    vocab_counter = collections.Counter(word for sent in sentences for word in sent)
    vocab = sorted(vocab_counter, key=vocab_counter.get, reverse=True)
    vocab.insert(PAD, '<PAD>')
    vocab.insert(SOS, '<SOS')
    w2id = {word:i for i, word in enumerate(vocab)}
    ids = [[w2id[word] for word in sent] for sent in sentences]
    return vocab_counter, vocab, w2id, ids

In [26]:
en_vocab_counter, en_vocab, en_w2id, en_ids = tokens2ids(en_tokens)
fr_vocab_counter, fr_vocab, fr_w2id, fr_ids = tokens2ids(fr_tokens)

In [27]:
len(en_vocab), len(fr_vocab)

(2803, 4345)

## Word vectors

FT English word vectors

In [28]:
import gensim.models.wrappers.fasttext
model = gensim.models.KeyedVectors.load_word2vec_format(ft_path, binary=False, encoding='utf8')
ft_model = model.wv
del model

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
ft_model.most_similar("dog") 

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('dogs', 0.856066107749939),
 ('puppy', 0.7839491963386536),
 ('Dog', 0.7767305374145508),
 ('canine', 0.7631831169128418),
 ('Mixed-breed', 0.7280029058456421),
 ('pet', 0.7213231325149536),
 ('terrier', 0.7139902114868164),
 ('labrador', 0.7112174034118652),
 ('puppies', 0.6918587684631348),
 ('non-dog', 0.6915143728256226)]

In [30]:
ft_model.vector_size

300

#### French word vectors

In [31]:
fr_w2v = KeyedVectors.load_word2vec_format(frWac200d_path)

In [32]:
fr_w2v.vector_size

300

Now we need to create embeddings matrices for english and french words of training corpus. If a word appears on GloVe or frWac then we load its pre-trained vector, otherwise we create a random vector.

In [33]:
def create_embedding(w2v, target_vocab, emb_dim):
    emb_len = len(target_vocab)
    embedding = np.zeros((emb_len, emb_dim))
    words_found = 0
    
    for i, w in enumerate(target_vocab):
        try: 
            embedding[i] = w2v[w]
            words_found += 1
        except KeyError:
            embedding[i] = np.random.normal(scale=0.6, size=(emb_dim, ))
    
    return embedding, words_found

In [34]:
en_emb, words_found = create_embedding(ft_model, en_vocab, 300)

In [35]:
en_emb.shape, words_found

((2803, 300), 2796)

In [36]:
fr_emb, words_found = create_embedding(fr_w2v, fr_vocab, 300)

In [37]:
fr_emb.shape, words_found

((4345, 300), 4101)

## Data preparation

Min, max and mean length of english sentences.

In [38]:
len_en_ids = [len(sentence) for sentence in en_ids]
min(len_en_ids), max(len_en_ids), np.mean(len_en_ids)

(3, 9, 6.035569393338994)

Min, max and mean length of french sentences.

In [39]:
len_fr_ids = [len(sentence) for sentence in fr_ids]
min(len_fr_ids), max(len_fr_ids), np.mean(len_fr_ids)

(2, 9, 6.196056231719973)

We set 30 as max length. In this example, we could use 10 since the max length of every sentence is 9. 30 allows us more flexibility in the future for other datasets.

In [40]:
maxlen = 30

In [41]:
en_train = pad_sequences(en_ids, maxlen, 'int64', 'post', 'post')
fr_train = pad_sequences(fr_ids, maxlen, 'int64', 'post', 'post')

In [42]:
fr_train.shape, en_train.shape, en_emb.shape, fr_emb.shape

((10599, 30), (10599, 30), (2803, 300), (4345, 300))

In [43]:
en_train[0]

array([3, 6, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [44]:
fr_train[0]

array([ 27,  30, 115,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

## Model

### Seq2Seq

<br/>Encoder:
<br/>
<br/>Inputs: french words sentence and initial hidden state (all zeros).
<br/>1- Look up at an embedding layer to get word vector of each word of the input sentence. 
<br/>2- Pass the word vectors sequence through a RNN.
<br/>3- Return hidden state of last timestep (vector representation of input sentence).
<br/>
<br/>Decoder:
<br/>
<br/>Inputs: 'SOS' word (i.e. start of sentence, is always the first word) and vector representation created by encoder.
<br/>1- Load vector representation as initial hidden state.
<br/>2- Look up at an embedding layer to get word vector of 'SOS'.
<br/>3- Pass the word vector through a RNN.
<br/>4- Generate prediction of next word.
<br/>5- Repeat 2, 3 and 4 using always the previous translated word until finish sentence translation.

In [45]:
torch.cuda.is_available()

True

In [46]:
fr_emb_t = torch.FloatTensor(fr_emb).cuda()
en_emb_t = torch.FloatTensor(en_emb).cuda()

In [47]:
def long_t(arr):
    return Variable(torch.LongTensor(arr)).cuda()

Load pre-trained vectors into an embedding layer.

In [48]:
def create_emb(emb_matrix, non_trainable=False):
    num_embeddings, embedding_dim = emb_matrix.size()
    emb = nn.Embedding(num_embeddings, embedding_dim)
    emb.load_state_dict({'weight': emb_matrix})
    if non_trainable:
        #emb.weight.requires_grad = False
        for param in emb.parameters():
            param.requires_grad = False
    return emb, num_embeddings, embedding_dim

Encoding layer

In [49]:
class EncoderRNN(nn.Module):
    def __init__(self, emb_matrix, hidden_size, num_layers=2):
        super(EncoderRNN, self).__init__()
        # Create embedding layer.
        self.embedding, num_embeddings, embedding_dim = create_emb(emb_matrix, True)
        # Create RNN.
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(embedding_dim, hidden_size, num_layers, batch_first=True)
        
    def forward(self, inp, hidden):
        return self.gru(self.embedding(inp), hidden)
    
    def init_hidden(self, batch_size):
        return Variable(torch.zeros(self.num_layers, batch_size, self.hidden_size))

Decoding layer

In [50]:
class DecoderRNN(nn.Module):
    def __init__(self, emb_matrix, hidden_size, num_layers=2):
        super(DecoderRNN, self).__init__()
        # Create embedding layer.
        self.emb, num_embeddings, embedding_dim = create_emb(emb_matrix)
        # Create RNN.
        self.gru = nn.GRU(embedding_dim, hidden_size, num_layers, batch_first=True, bidirectional=False)
        self.out = nn.Linear(hidden_size, num_embeddings)
        
    def forward(self, inp, hidden):
        emb = self.emb(inp).unsqueeze(1)
        res, hidden = self.gru(emb, hidden)
        # Softmax layer, generates probs for each word vector of the embedding layer.
        res = F.log_softmax(self.out(res[:,0]), dim=1)
        return res, hidden

In [51]:
def encode(inp, encoder):
    batch_size, input_length = inp.size()
    hidden = encoder.init_hidden(batch_size).cuda()
    enc_outputs, hidden = encoder.forward(inp, hidden)
    return long_t([SOS]*batch_size), enc_outputs, hidden

Training

We use teaching-force as training approach. Rather than pass to decoder the previous translated word, we pass the real target.

In [52]:
def fit(encoder, decoder, train_dl, n_epochs, enc_optim, dec_optim, criterion):
    bar = tqdm_notebook(total=n_epochs)
    loss_tracker = []
    avg_mom = 0.98
    avg_loss = 0.
    batch_num = 0

    for epoch in range(n_epochs):
        bar2 = tqdm_notebook(total=train_dl.dataset.shape[0] / train_dl.batch_size, desc=f'Epoch {epoch}', leave=False)
        for i, batch in enumerate(train_dl):
            batch_num += 1
            loss = 0
            
            inp = long_t(batch[:, :maxlen])

            targ = long_t(batch[:, maxlen:])
       
            # Encoder creates a vector representation of input french sentence. 
            decoder_input, encoder_output, hidden = encode(inp, encoder)

            # Zero the gradients before running the backward pass.
            enc_optim.zero_grad()
            dec_optim.zero_grad()
            
            targ_length = targ.size()[1]
     
            for di in range(targ_length):
                decoder_output, hidden = decoder(decoder_input, hidden)
                # Teacher forcing: the decoder receives as input the real target instead of predicted word.
                decoder_input = targ[:, di]
                
                # Compute loss.
                loss += criterion(decoder_output, decoder_input)
          
            # Backward pass: compute gradient of the loss with respect to all the learnable parameters of the model.
            loss.backward()

            # Calling the step function on an Optimizer makes an update to its parameters.
            enc_optim.step()
            dec_optim.step()
           
            # Exponentially weighted moving average, to make the reported loss more stable.
            avg_loss = avg_loss * avg_mom + (loss.data.item() / targ_length)  * (1-avg_mom)
            
            # Compute bias-corrected loss estimate.
            debias_loss = avg_loss / (1 - avg_mom**batch_num)
            
            bar2.update()
            
        loss_tracker.append(np.round([epoch, debias_loss], 6))
        print(np.round([epoch, debias_loss], 6))    
        bar.update()
    return loss_tracker

In [53]:
def req_grad_params(o):
    return (param for param in o.parameters() if param.requires_grad)

Initialize models and set parameters

In [54]:
hidden_size = 64 #128
encoder = EncoderRNN(fr_emb_t, hidden_size).cuda()
decoder = DecoderRNN(en_emb_t, hidden_size).cuda()

In [55]:
lr = 0.0001

In [56]:
enc_opt = optim.Adam(req_grad_params(encoder), lr=lr)
dec_opt = optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.NLLLoss().cuda()

In [57]:
batch_size = 64
#64

Create a dataloader

In [ ]:
train_dl = DataLoader(np.concatenate([fr_train, en_train], 1), batch_size, shuffle=True, num_workers=1)

Train the model

In [ ]:
loss_tracker = fit(encoder, decoder, train_dl, 200, enc_opt, dec_opt, criterion)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


[0.       0.141475]


[1.       0.134318]


[2.      0.12782]


...

[1.9500e+02 8.3855e-02]


[1.9600e+02 8.4194e-02]


[1.9700e+02 8.2341e-02]


[1.9800e+02 8.3255e-02]


[1.9900e+02 8.2474e-02]


In [ ]:
loss_tracker

[array([0.      , 0.141475]),
 array([1.      , 0.134318]),
 array([2.     , 0.12782]),
 array([3.      , 0.123532]),
 array([4.      , 0.120557]),
 array([5.      , 0.120755]),
 array([6.      , 0.119089]),
 array([7.      , 0.116263]),
 array([8.      , 0.115754]),
 array([9.      , 0.114742]),
 array([10.      ,  0.113279]),
 array([11.      ,  0.112437]),
 array([12.      ,  0.109533]),
 array([13.      ,  0.110783]),
 array([14.      ,  0.108534]),
 array([15.      ,  0.110208]),
 array([16.      ,  0.108641]),
 array([17.      ,  0.108041]),
 array([18.      ,  0.109272]),
 array([19.      ,  0.107649]),
 array([20.     ,  0.10606]),
 array([21.      ,  0.106915]),
 array([22.      ,  0.106317]),
 array([23.      ,  0.104948]),
 array([24.      ,  0.104234]),
 array([25.      ,  0.104382]),
 array([26.      ,  0.103635]),
 array([27.      ,  0.102623]),
 array([28.      ,  0.102079]),
 array([29.      ,  0.104037]),
 array([30.      ,  0.102396]),
 array([31.      ,  0.102229]),


In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/NLP/Machine Translation/FT_Files/loss_tracker_FT_200E_.084Loss.pkl', 'wb') as f:
  pickle.dump(loss_tracker, f)

Result trackers:   
  
100 Epochs: 0.1384 Loss


In [ ]:
torch.save(encoder.state_dict(), f'/content/drive/My Drive/Colab Notebooks/NLP/Machine Translation/FT_Files/FTsimple_encoder_200E_0.084Loss.pth')
torch.save(decoder.state_dict(), f'/content/drive/My Drive/Colab Notebooks/NLP/Machine Translation/FT_Files/FTsimple_decoder_200E_0.084Loss.pth')

# New Section

Load model weights

In [58]:
encoder.load_state_dict(torch.load(encoder_saved_model_weights))

<All keys matched successfully>

In [59]:
decoder.load_state_dict(torch.load(decoder_saved_model_weights))

<All keys matched successfully>

In order to generate predictions of a french sentence:
<br/>1- Tokenize.
<br/>2- Transform words to their ids.
<br/>3- Set sentence length = 30.
<br/>3- Encode.
<br/>4- Decode next translated word until the decoder generates a special word that means end of sentence or until reach the max length = 30.

In [60]:
def sent2ids(sent):
    ids = [fr_w2id[t] for t in tokenize(sent)]
    return pad_sequences([ids], maxlen, 'int64', 'post', 'post')
  

In [61]:
def evaluate(inp):
    decoder_input, encoder_outputs, hidden = encode(inp, encoder)
    target_length = maxlen
    
    decoded_words = []
    for di in range(target_length):
        decoder_output, hidden = decoder(decoder_input, hidden)
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]
        if ni==PAD:
            break
        decoded_words.append(en_vocab[ni])
        decoder_input = long_t([ni])
    
    return decoded_words

In [62]:
def fr2en(sent):
    ids = long_t(sent2ids(sent))
    translation = evaluate(ids)
    return ' '.join(translation)

Bleu

In [63]:
from nltk.translate.bleu_score import sentence_bleu
# returns the one gram bleu score 

def bleu(reference,candidate):
  one_gram = sentence_bleu([reference], candidate, weights=(1, 0, 0, 0))
  return(one_gram)

In [64]:
from nltk.translate.gleu_score import sentence_gleu

def gleu(reference, candidate):
  one_gram = sentence_gleu(reference, candidate)
  return (one_gram)

Evaluate n random pairs with bleu

In [65]:
pairs = [[fr_questions[idx],en_questions[idx]] for idx,i in enumerate(sentences) ]

In [66]:
import random
import warnings
warnings.filterwarnings("ignore")

# evaluate n random sentence pairs using one gram bleu

def evaluateRandomly(encoder, decoder, n=25):
    score_tracker = []
    for i in range(n):
        pair = random.choice(pairs)
        print(i+1)

        output_words = fr2en(pair[0])
        output_sentence = ''.join(output_words)
        
        #bleu
        ref = pair[1].split()[:-1]
        pred = output_sentence.split()[:-1]
        ref, pred = fix_contractions(ref, pred)

        print('>', pair[0])
        print('=', ref)
        print('<', pred,'\n')
        one_gram = bleu(ref,pred)
        score_tracker.append(one_gram)
        print(f'Bleu Score: {one_gram}')
        print('')

    print('Avg Bleu Score (based on one-gram): ',sum(score_tracker)/len(score_tracker))

In [67]:
# sometimes ending punctuation is filtered off prediction when no EOS token is predicted
# adding it back in to not trigger missed prediction
# input: two lists of words
# output: two lists of words

def fix_punctuation(ref, pred):
  ending_punc = [ref[-1]]
  if pred[-1] not in ending_punc:
    pred.append(ending_punc[0])
  return ref, pred

In [68]:
import gensim
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/Colab Notebooks/NLP/Machine Translation/FT_Files/word2vec.bin', binary=True)

In [69]:
# create a custom score by averaging bs and gs scores with weights
# adding a bonus if different words have a shared semantic meaning (cs score > 0.3)
# subtracting points if predicted sequence has duplicated words
# inputs: bleu score, gleu score, # of double words in pred
# outputs: custom score

def custom_score(bs,gs,cs,double_word_penalty, verbose=True):
  total = ((bs*.75)+(gs*.25)) # weighted avg
  cs_bonus = 0

  # calc cs bonus
  def get_bonus(cs,multiplier = .4):
    additional = 0
    for i in cs:
      additional += i * multiplier
    return additional

  # if we have similarities, compute bonus
  if cs: 
    cs_bonus = get_bonus(cs)


  # if perfect score, return 1
  if bs == 1:
    if verbose:
      print('\nSemantic similarity bonus : +', float(cs_bonus))
      print('Double word penalty:        -', double_word_penalty * .1,'\n')
    return 1.00

  else:
    if cs_bonus:
      grand_total = total + cs_bonus
      if grand_total < 1:
        if verbose:
          print('\nSemantic similarity bonus : +', float(cs_bonus))
          print('Double word penalty:        -', double_word_penalty * .1,'\n')
        return grand_total

      # bonus put score over 1  
      else:
        cs_bonus = get_bonus(cs, multiplier = .3)
        if verbose:
          print('\nTotal score > 1, adjusting weights...') #debug statement, delete at end
          print('\nSemantic similarity bonus : +', float(cs_bonus))
          print('Double word penalty:        -', double_word_penalty * .1,'\n')
        grand_total = total + cs_bonus
        if grand_total < 1:   
          return grand_total

        # bonus put score over 1   
        else:
          cs_bonus = get_bonus(cs, multiplier = .2)
          grand_total = total + cs_bonus
          if grand_total < 1:  
            return grand_total
          else:
            cs_bonus = get_bonus(cs, multiplier = .1)
            grand_total = total + cs_bonus
            if grand_total < 1:   
              return grand_total



    # if no cs bonus      
    else:
      if verbose:
        print('\nSemantic similarity bonus : +', float(cs_bonus))
        print('Double word penalty:        -', double_word_penalty * .1,'\n')
      return total - (double_word_penalty * .1)

Compare Ref to Pred

In [70]:
# calculates penalties for double words, filters sentences to relevant words to compare,
# calculateds cos. sim., and a score for the strength of the cos. sims.
# inputs: two lists of words
# outputs: relevant word cosine sim. scores over 0.3, number of double words in the prediction

def similarities(A,B,verbose=True):

  # does B have more double words than A? 
  doublesA = 0
  doublesB = 0
  basketA = []
  basketB = []

  for i in A:
    if i not in basketA:
      basketA.append(i)
    else:
      doublesA += 1

  for i in B: 
    if i not in basketB:
      basketB.append(i)
    else:
      doublesB += 1

  # calc penalty, keep only positive values
  double_word_penalty = np.clip(doublesB - doublesA, 0,3) 

  # get words not in the other sentence and not in punc/stopwords
  stop_words = ['a','an','of','the','to','on','t','in','as'] #,'not','no']
  punc = ['.','?','!',',']
  extraW = [] # all extra words
  extraA = []
  extraB = []

  for i in A:
    if i not in punc:
      if i not in stop_words:
        if (i not in B):
          extraA.append(i)
  for i in B:
    if i not in punc:
      if i not in stop_words:
        if (i not in A):
          extraB.append(i)

  extraW = extraA + extraB

  # if off by one word, exit
  if len(extraW) == 1:
    return [0, double_word_penalty]
  
  # calc cos sims and score
  sim_finn = []
  sim_w2v = []
  record = []

  for a, b in itertools.product(extraA,extraB):
    sim_w2v.append([a,b,w2v_model.similarity(a,b)])

  sorted_sim_w2v = sorted(sim_w2v, key = lambda x: x[2], reverse=True)

  cs_score = [0] #list of cs over 0.3

  # print cs scores
  if sorted_sim_w2v:
    if verbose:
      print('\nSemantic similarities using w2v:')
    for idx,i in enumerate(sorted_sim_w2v):
      if verbose:
        if i[2] > 0.3:
          print(bold, end="")
          print('  ',i,reset)
        else:
          print('  ',i)
    #print('\n')
    # record cs scores
    for i in sorted_sim_w2v:
      if i[2] > 0.3:
        cs_score.append(i[2])

  return [cs_score, double_word_penalty]

In [91]:
# evaluate n random pairs from dataset
# input: models, n
# output: none

def evaluateRandomly(encoder, decoder, n=100, all = False):
    print(bold+'Evaluation of Machine Translation Model'+reset)
    
    bleu_score_tracker = []
    gleu_score_tracker = []
    custom_tracker = []
    record_test = []
    ending_punc = ['.','?','!']
    glove_frwac_ref_pred = []

    if not all: # run on n randomly chosen pairs
      print(bold+'Evaluating ',n,' examples...'+reset)
      for i in range(n):
          pair = random.choice(pairs)

          output_words = fr2en(pair[0])
 
          # lists
          ref = pair[1].split()#[:-1]
          pred = output_words.split()[:-1]

          # fix contractions
          ref, pred = fix_contractions(ref, pred)

          # if missing ending punctuation
          if pred[-1] not in ending_punc:
            ref, pred = fix_punctuation(ref, pred) 

          glove_frwac_ref_pred.append([i,ref,pred])

          #print('Before Bleu: ',ref[:-1],pred[:-1])
          bleu_one_gram = bleu(ref[:-1],pred[:-1])


          # DO NOT DISPLAY PERFECT SCORES - USED FOR EASY DEBUGGING - DELETE AT END
          # CONVERT TO DISPLAYING PERFECT SCORES SOME FRACTION OF THE TIME (1/5TH?)
          if bleu_one_gram:
            print('\n')
            print(bold+'Input:\t'+reset, pair[0])
            print(bold+'Target:\t'+reset, ' '.join(ref))

            # GOOGLE TRANSLATE FUNCTION - ALLOTED LIMITED TRANSLATIONS PER DAY
            #gt = google_translate(pair[0])
            #gt = normalizeString(gt)
            #gt, _ = fix_contractions(gt.split(),' ')
            #gt = ' '.join(gt)
            #print('GT:\t',gt)
            #bleu_one_gram_gt = bleu([ref[:-1]],gt[:-1])
            #print('GT Bleu Score: ',bleu_one_gram_gt)
            #if bleu_one_gram > bleu_one_gram_gt:
              #print('Better than GT!')

            print(bold+'Pred:\t'+reset, ' '.join(pred),'\n')
            
            # requires ref to be a 2d list, pred 1d list
            bleu_score_tracker.append(bleu_one_gram)
            print(f'Bleu Score: {bleu_one_gram:.3f}')

            gleu_one_gram = gleu([ref[:-1]],pred[:-1])
            gleu_score_tracker.append(gleu_one_gram)
            print(f'Gleu Score: {gleu_one_gram:.3f}')
            print(f'Avg Score:  {(gleu_one_gram*.25+bleu_one_gram*.75):.3f}') #weighted
            
            cs_score = 0

            # if not perfect score: calc. bonuses and penalties
            if bleu_one_gram < 1:
              try: # sometimes sims returns none
                sim_returns = similarities(ref,pred)
                cs_score = sim_returns[0]
                double_word_penalty = sim_returns[1]
                cust_score = custom_score(bleu_one_gram,gleu_one_gram,cs_score,double_word_penalty)
                print(f'{bold_red_font_tag}Custom Score: {cust_score:.3f}{reset}')
                #print('\n')

              # if word not in WE
              except KeyError:
                print('Cosine similarities: Word not found in embedding vocabulary')
                continue
            else:
              cust_score = custom_score(bleu_one_gram,gleu_one_gram,0,0)
              print(f'{bold_red_font_tag}Custom Score: {cust_score:.3f}{reset}')

            custom_tracker.append(cust_score)

      print('\n')
      print(f'{bold_blue_font_tag}Avg Bleu Score  :{reset} {sum(bleu_score_tracker)/len(bleu_score_tracker):.3f}')
      print(f'{bold_blue_font_tag}Avg Gleu Score  :{reset} {sum(gleu_score_tracker)/len(gleu_score_tracker):.3f}')
      print(f'{bold_blue_font_tag}Avg Custom Score:{reset} {sum(custom_tracker)/len(custom_tracker):.3f}')

    else: # run on entire dataset
      print(bold+'Evaluating entire dataset...'+reset)
      for i in range(len(pairs)):

        
        pair = pairs[i]
        output_words = fr2en(pair[0])
        ref = pair[1].split()#[:-1]
        pred = output_words.split()[:-1]

        # fix contractions
        ref, pred = fix_contractions(ref, pred)

        # if missing ending punctuation
        if pred[-1] not in ending_punc:
          ref, pred = fix_punctuation(ref, pred) 

        glove_frwac_ref_pred.append([i,ref,pred])

        bleu_one_gram = bleu(ref[:-1],pred[:-1])
        bleu_score_tracker.append(bleu_one_gram)
        gleu_one_gram = gleu([ref[:-1]],pred[:-1])
        gleu_score_tracker.append(gleu_one_gram)

        cs_score = 0

        # if not perfect score: calc. bonuses and penalties
        if bleu_one_gram < 1:
          try: # sometimes sims returns none
            sim_returns = similarities(ref,pred, verbose=False)
            cs_score = sim_returns[0]
            double_word_penalty = sim_returns[1]
            cust_score = custom_score(bleu_one_gram,gleu_one_gram,cs_score,double_word_penalty, verbose=False)
            #print(bold_red_font_tag+'Custom Score: ',cust_score,reset)
            #print('\n')

          # if word not in WE
          except KeyError:
            #print('Cosine similarities: Word not found in embedding vocabulary')
            continue
        else:
          cust_score = custom_score(bleu_one_gram,gleu_one_gram,0,0, verbose=False)
          #print(bold_red_font_tag+'Custom Score: ',cust_score,reset)

        custom_tracker.append(cust_score)
        
      '''print(len(custom_tracker))
      
      print(type(custom_tracker))
      print(type(custom_tracker[1]))'''
      print('\n')
      custom_tracker = list(filter(None, custom_tracker)) 
      #custom_tracker=[float(i) for i in custom_tracker]
      print(f'{bold_blue_font_tag}Avg Bleu Score  :{reset} {sum(bleu_score_tracker)/len(bleu_score_tracker):.3f}')
      print(f'{bold_blue_font_tag}Avg Gleu Score  :{reset} {sum(gleu_score_tracker)/len(gleu_score_tracker):.3f}')
      print(f'{bold_blue_font_tag}Avg Custom Score:{reset} {sum(custom_tracker)/len(custom_tracker):.3f}')

    return glove_frwac_ref_pred

Evaluate bleu score on entire on all training data

In [72]:
# get bleu score for entire dataset

def bleuScore(encoder, decoder):
  score_tracker = []
  for i in range(len(pairs)):
      pair = pairs[i]
      #print('pair: ',pair)
      output_words = fr2en(pair[0])
      #print(output_words)
      output_sentence = output_words
      ref = pair[1].split()[:-1]
      pred = output_sentence.split()[:-1]
      ref, pred = fix_contractions(ref, pred)
      #print(ref)
      #print(pred)
      one_gram = bleu(ref,pred)
      score_tracker.append(one_gram)


  print('Avg Bleu Score: ',sum(score_tracker)/len(score_tracker))

    

In [73]:
'''
def fix_contractions(ref,pred):
  for idx, word in enumerate(pred):
    if word == 're':
      pred[idx] = 'are'
    elif word == 'm':
      pred[idx] = 'am' 
    elif word == 's':
      pred[idx] = 'is'   
    elif word == 'aren': 
      try:
        if pred[idx+1] == 't':
          pred[idx] = 'are' 
          pred[idx+1] = 'not'
      except IndexError:
        continue

  for idx, rword in enumerate(ref):
    if rword == 're':
      ref[idx] = 'are'
    elif rword == 'm':
      ref[idx] = 'am' 
    elif rword == 's':
      ref[idx] = 'is'  
    elif rword == 'aren': 
      try:
        if ref[idx+1] == 't':
          ref[idx] = 'are' 
          ref[idx+1] = 'not'      
      except IndexError:
        continue
        
  return ref, pred'''

"\ndef fix_contractions(ref,pred):\n  for idx, word in enumerate(pred):\n    if word == 're':\n      pred[idx] = 'are'\n    elif word == 'm':\n      pred[idx] = 'am' \n    elif word == 's':\n      pred[idx] = 'is'   \n    elif word == 'aren': \n      try:\n        if pred[idx+1] == 't':\n          pred[idx] = 'are' \n          pred[idx+1] = 'not'\n      except IndexError:\n        continue\n\n  for idx, rword in enumerate(ref):\n    if rword == 're':\n      ref[idx] = 'are'\n    elif rword == 'm':\n      ref[idx] = 'am' \n    elif rword == 's':\n      ref[idx] = 'is'  \n    elif rword == 'aren': \n      try:\n        if ref[idx+1] == 't':\n          ref[idx] = 'are' \n          ref[idx+1] = 'not'      \n      except IndexError:\n        continue\n        \n  return ref, pred"

In [74]:
# fix issues with contractions when displaying results.
# issues: 's' could represent possesion and not 'is.' Small fraction of the time though.
# inputs: two lists of words
# outputs: two lists of words

def fix_contractions(ref,pred):

  for idx, word in enumerate(pred):
    if word == 're':
      pred[idx] = 'are'
    elif word == 'm':
      pred[idx] = 'am' 
    elif word == 's':
      pred[idx] = 'is'   
    elif word == 'ok':
      pred[idx] = 'okay'  
    elif word == 'aren': 
      try:
        if pred[idx+1] == 't':
          pred[idx] = 'are' 
          pred[idx+1] = 'not'
      except IndexError:
        continue

    elif word == 'isn': 
      try:
        if pred[idx+1] == 't':
          pred[idx] = 'is' 
          pred[idx+1] = 'not'
      except IndexError:
        continue
    elif (word == 'don' and pred[idx+1] == 't'):
      pred[idx] = 'do' 
      pred[idx+1] = 'not'

  for idx, rword in enumerate(ref):
    if rword == 're':
      ref[idx] = 'are'
    elif rword == 'm':
      ref[idx] = 'am' 
    elif rword == 'ok':
      ref[idx] = 'okay'       
    elif rword == 's':
      ref[idx] = 'is'  
    elif rword == 'aren': 
      try:
        if ref[idx+1] == 't':
          ref[idx] = 'are' 
          ref[idx+1] = 'not'      
      except IndexError:
        continue    
    elif rword == 'isn': 
      try:
        if ref[idx+1] == 't':
          ref[idx] = 'is' 
          ref[idx+1] = 'not'      
      except IndexError:
        continue   
    elif (rword == 'don' and ref[idx+1] == 't'):
      ref[idx] = 'do' 
      ref[idx+1] = 'not' 

  return ref, pred

In [75]:
bold_blue_font_tag = '\x1b[1m\x1b[34m'
bold_red_font_tag = '\x1b[1m\x1b[31m'
bold_gree_font_tag = '\x1b[1m\x1b[32m'
magenta = '\033[35m'
bold = '\033[1m'
reset = '\033[0m'

Evaluate on a sample

In [76]:
ft_frwac_ref_pred = evaluateRandomly(encoder, decoder, all= False)

Evaluation of Machine Translation Model
Evaluating  100  examples...


Input:	 nous essayons .
Target:	 we are trying .
Pred:	 we are trying . 

Bleu Score: 1.000
Gleu Score: 1.000
Avg Score:  1.000

Semantic similarity bonus : + 0.0
Double word penalty:        - 0.0 

Custom Score: 1.000


Input:	 je vais bien .
Target:	 i am okay .
Pred:	 i am fine . 

Bleu Score: 0.667
Gleu Score: 0.500
Avg Score:  0.625

Semantic similarities using w2v:
   ['okay', 'fine', 0.38318834] 

Semantic similarity bonus : + 0.1532753348350525
Double word penalty:        - 0.0 

Custom Score: 0.778


Input:	 tu es une etudiante .
Target:	 you are a student .
Pred:	 you are a terrible person . 

Bleu Score: 0.600
Gleu Score: 0.429
Avg Score:  0.557

Semantic similarities using w2v:
   ['student', 'person', 0.33660564] 
   ['student', 'terrible', 0.05019008]

Semantic similarity bonus : + 0.1346422553062439
Double word penalty:        - 0.0 

Custom Score: 0.692


Input:	 je suis toujours occupe .
Target:	 i 

In [92]:
ft_frwac_ref_pred = evaluateRandomly(encoder, decoder, all= True)

Evaluation of Machine Translation Model
Evaluating entire dataset...


Avg Bleu Score  : 0.755
Avg Gleu Score  : 0.670
Avg Custom Score: 0.803


In [93]:
len(ft_frwac_ref_pred)

10599

Save results to pickle file for scratch notebook - Travis needs this file

In [94]:
with open('/content/drive/My Drive/Colab Notebooks/NLP/Machine Translation/FT_Files/ft_frwac_results_V1_200Epochs_.084Loss.pkl', 'wb') as f:
  pickle.dump(ft_frwac_ref_pred, f)